# Scenerio Builder
*Use this notebook to generate new scenarios for the simulation.*

In [8]:
import GWorld
import pprint
import json
import numpy as np

The Scenario Buiilder creates new scenarios using already existing scenarios as a template.

## Inputs
- `Scenario_source_file` : Name of the JSON file which has the **template scenario**
- `Scenario_destination_file` : Name of the JSON file to which the **new scenario** is to be saved. 
- `Template_Scenario` : Name of the **template scenario** in `Scenario_source_file` which is to be used as the template for the **new scenario**
- `new_scenario_name` : Name of the **new scenario**

In [23]:
# For creating a New Scenarion based on Some Scenario
Template_Scenario = 'SingleLane20-8Agents'
new_scenario_name = 'SingleLane10-4Agents'

Scenario_source_file = 'Scenarios4FeARSims.json'
Scenario_destination_file = 'Scenarios4FeARSims.json'

Scenario = GWorld.LoadJsonScenario(json_filename=Scenario_source_file,scenario_name=Template_Scenario)

# Converting back to JSON format
Scenario= json.loads(json.dumps(Scenario,indent=1, sort_keys=True))

# View Template Scenario

In [24]:
# View  Template Scenario
pretty_print_json = pprint.pformat(Scenario,width=150).replace("'", '"')
print(pretty_print_json)

{"AgentLocations": [],
 "DirectionWeights": [0, 0, 0, 1],
 "Map": {"OneWays": [],
         "Region": [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]],
         "Walls": []},
 "N_Agents": 8,
 "N_Cases": 10000,
 "N_iterations": 1,
 "SpecificAction4Agents": [],
 "SpecificDirectionWeights4Agents": [],
 "SpecificStepWeights4Agents": [],
 "StepWeights": [1, 1, 1, 1, 1],
 "defaultAction": "random"}


---


# Generate New Map
The following commands can be useful in defining a NewMap for the scenario

- The Map of the GWorld Scenario is represented by a Matrix of ones and zeros. 
- **Ones** represent **valid locations** the agents in the simulation can occupy.
- **Zeros** represent **invalid locations which cannot be occupied by agents.

In [25]:
# # NewMap = np.ones([5,10])
# NewMap = np.zeros([5,10])
NewMap = np.ones([1,10])

# # NewMap[0,:] = 1
# NewMap[1,:] = 1
# NewMap[2,:] = 1

# # NewMap[:,0] = 1
# # NewMap[:,9] = 1
# # NewMap[:,4] = 1
# # NewMap[:,5] = 1
# # NewMap[:,2] = 1
# NewMap[:,7] = 1
# NewMap[:,8] = 1


print(NewMap)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


# Make changes to the values of the Template

- `Overwrite = False` ensures that already existing scenarios are not overwritten.

In [28]:
# Making Changes

# Overwrite = False
Overwrite = True


# Scenario['Map']['OneWays'] = []
# Scenario['Map']['Walls'] = []
Scenario['Map']['Region'] = NewMap.tolist()
Scenario['DirectionWeights'] = [0,0,0,1]
# Scenario['StepWeights'] = [1,1,1,1,1]
# Scenario['AgentLocations'] = [[2,3],[2,5],[1,7]]
Scenario['N_Agents'] = 4
Scenario['N_iterations'] = 1
Scenario['N_Cases'] = 10000
# Scenario['SpecificStepWeights4Agents'] = []
# Scenario['SpecificDirectionWeights4Agents'] = [[[2],[0,1,0,0]]]


# Review New Scenario

In [29]:
# Pretty Formating

pretty_print_json = pprint.pformat(Scenario,width=150).replace("'", '"')
print(pretty_print_json)

{"AgentLocations": [],
 "DirectionWeights": [0, 0, 0, 1],
 "Map": {"OneWays": [], "Region": [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], "Walls": []},
 "N_Agents": 4,
 "N_Cases": 10000,
 "N_iterations": 1,
 "SpecificAction4Agents": [],
 "SpecificDirectionWeights4Agents": [],
 "SpecificStepWeights4Agents": [],
 "StepWeights": [1, 1, 1, 1, 1],
 "defaultAction": "random"}


In [30]:
# Add Scenario to JSON file

GWorld.AddJsonScenario(json_filename=Scenario_destination_file,new_scenario=Scenario, new_scenario_name=new_scenario_name, Overwrite = Overwrite)

Overwriting Scenario:  SingleLane10-4Agents


True